In [ ]:
#https://www.kaggle.com/boojum/connecting-voxel-spaces/

I thought that all pipelines presented on public notebooks are giving random output, so I decided to think a little bit differently. The idea is to extract features using MRI images and tumor mask. Here I'm giving an example for one image, maybe I'm doing something wrong, but I'll be glad to get feedback from the community. Also I'm worried that it will be out of time on the whole test set

### Registration 
https://www.kaggle.com/boojum/connecting-voxel-spaces/
![](https://sun9-45.userapi.com/impg/Flbnug2OUli1ecXsoIKeUasIGXGj_5hqjX4cRg/z2nfz8-b3a0.jpg?size=2560x1153&quality=96&sign=0536543610f1655d967af88dbc775e98&type=album)

### Segmentation 
Unet
https://pytorch.org/hub/mateuszbuda_brain-segmentation-pytorch_unet/

### Features

https://pyradiomics.readthedocs.io/en/latest/features.html#module-radiomics.shape2D

# EJERCICIO RADIÓMICA
## Ángel Guevara y Arturo Sirvent

In [ ]:
!pip install pyradiomics

In [ ]:
import os
import sys 
from tqdm import tqdm
import numpy as np
import pandas as pd
from PIL import Image
import pydicom
import torch
import nibabel as nib
import matplotlib.pyplot as plt
import SimpleITK as sitk
import radiomics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import plot_roc_curve, classification_report


train_path = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/'
train_label_path = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv'

In [ ]:
train_dirs = sorted(os.listdir(train_path))

In [ ]:
reader = sitk.ImageSeriesReader()
reader.LoadPrivateTagsOn()

In [ ]:
def resample(image, ref_image):

    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(ref_image)
    resampler.SetInterpolator(sitk.sitkLinear)
    
    resampler.SetTransform(sitk.AffineTransform(image.GetDimension()))

    resampler.SetOutputSpacing(ref_image.GetSpacing())

    resampler.SetSize(ref_image.GetSize())

    resampler.SetOutputDirection(ref_image.GetDirection())

    resampler.SetOutputOrigin(ref_image.GetOrigin())

    resampler.SetDefaultPixelValue(image.GetPixelIDValue())

    resamped_image = resampler.Execute(image)
    
    return resamped_image

In [ ]:
def normalize(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [ ]:
%%time
def get_img(index):
    filenamesDICOM = reader.GetGDCMSeriesFileNames(f'{train_path}/{train_dirs[index]}/T1w')
    reader.SetFileNames(filenamesDICOM)
    t1_sitk = reader.Execute()

    filenamesDICOM = reader.GetGDCMSeriesFileNames(f'{train_path}/{train_dirs[index]}/FLAIR')
    reader.SetFileNames(filenamesDICOM)
    flair_sitk = reader.Execute()

    filenamesDICOM = reader.GetGDCMSeriesFileNames(f'{train_path}/{train_dirs[index]}/T1wCE')
    reader.SetFileNames(filenamesDICOM)
    t1wce_sitk = reader.Execute()

    flair_resampled = resample(flair_sitk, t1_sitk)
    t1wce_resampled = resample(t1wce_sitk, t1_sitk)

    t1_sitk_array = normalize(sitk.GetArrayFromImage(t1_sitk))
    flair_resampled_array = normalize(sitk.GetArrayFromImage(flair_resampled))
    t1wce_resampled_array = normalize(sitk.GetArrayFromImage(t1wce_resampled))

    stacked = np.stack([t1_sitk_array, flair_resampled_array, t1wce_resampled_array])

    to_rgb = stacked[:,t1_sitk_array.shape[0]//2,:,:].transpose(1,2,0)
    im = Image.fromarray((to_rgb * 255).astype(np.uint8))
    return im

In [ ]:
model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=3, out_channels=1, init_features=32, pretrained=True)

## Carga y visualización de los datos

In [ ]:
# Nos quedamos con las primeras 80 imágenes
im = []
for i in range(80):
    im.append(get_img(i))


In [ ]:
# Cargamos las etiquetas
labels_train = pd.read_csv(train_label_path)[0:80]
labels_train=labels_train.set_index('BraTS21ID')
labels_train

In [ ]:
# Aplicamos el modelo de segmentación a las imágenes que nos hemos guardado
test_img=[]
test_res=[]
for i in range(80):
    test_img.append(np.array([np.moveaxis(np.array(im[i].resize((256, 256))), -1, 0)]))
    test_res.append(model(torch.Tensor(test_img[i])))

In [ ]:
# Mostramos las primeras 20 imágenes. Las 3 primeras columnas son las imagénes de las tres pruebas que se realizan mientras 
# que la última es la segmentación a partir de esas imágenes.
# En cada fila tenemos una muestra.
plt.figure(figsize=(20,70))
for i in range(20):
    plt.subplot(20,4,1+i*4)
    plt.imshow(test_img[i][0, 0])
    plt.title('Prueba1')
    plt.xticks([])
    plt.yticks([])
    plt.subplot(20,4,2+i*4)
    plt.imshow(test_img[i][0, 1])
    plt.title('Prueba2')
    plt.xticks([])
    plt.yticks([])
    plt.subplot(20,4,3+i*4)
    plt.imshow(test_img[i][0, 2])
    plt.title('Prueba3')
    plt.xticks([])
    plt.yticks([])
    plt.subplot(20,4,4+i*4)
    plt.imshow(test_res[i].detach().numpy()[0,0])
    plt.title('Segmentación')
    plt.xticks([])
    #plt.title()
    plt.yticks([])

plt.tight_layout()

## Extracción de características

Ahora, extraemos diferentes características de las imágenes para poder construir un clasificador. 

In [ ]:
# Características de Forma 2D
shape=[]
for i in range(80):
    aux=radiomics.shape2D.RadiomicsShape2D(
        sitk.GetImageFromArray(test_img[i]), 
        sitk.GetImageFromArray(np.array([
            test_res[i][0][0].detach().cpu().numpy() > 0.5
        ]).astype(np.uint8)),
        force2D=True
    )
    shape.append(aux)

In [ ]:
MeshSurface=[]
PixelSurface=[]
Perimeter=[]
PerimeterSurfaceRatio=[]
Spericity=[]
SphericalDisproportion=[]
MaximumDiameter=[]
MajorAxisLength=[]
MinorAxisLenth=[]
Elongation=[]

for i in range(80):
    MeshSurface.append(shape[i].getMeshSurfaceFeatureValue())
    PixelSurface.append(shape[i].getPixelSurfaceFeatureValue())
    Perimeter.append(shape[i].getPerimeterFeatureValue())
    PerimeterSurfaceRatio.append(shape[i].getPerimeterSurfaceRatioFeatureValue())
    Spericity.append(shape[i].getSphericityFeatureValue())
    SphericalDisproportion.append(shape[i].getSphericalDisproportionFeatureValue())
    MaximumDiameter.append(shape[i].getMaximumDiameterFeatureValue())
    MajorAxisLength.append(shape[i].getMajorAxisLengthFeatureValue())
    MinorAxisLenth.append(shape[i].getMinorAxisLengthFeatureValue())
    Elongation.append(shape[i].getElongationFeatureValue())


In [ ]:
# Construimos un dataframe con las características de forma
df = pd.DataFrame()
df['ID'] = [int(i) for i in train_dirs][0:80]
df = df.set_index('ID')
df['MeshSurface'] = MeshSurface
df['PixelSurface'] = PixelSurface
df['Perimeter'] = Perimeter
df['PerimeterSurfaceRatio'] = PerimeterSurfaceRatio
df['Spericity'] = Spericity
df['SphericalDisproportion'] = SphericalDisproportion
df['MaximumDiameter'] = MaximumDiameter
df['MajorAxisLength'] = MajorAxisLength
df['MinorAxisLenth'] = MinorAxisLenth
df['Elongation'] = Elongation
#df = df.join(labels_train)
df

In [ ]:
# Características de Textura (GLCM)
texturas=[]
for i in range(80):
    aux=radiomics.glcm.RadiomicsGLCM(
        sitk.GetImageFromArray(test_img[i][0,0,:,:].reshape(1, 256, 256)), 
        sitk.GetImageFromArray(np.array([
            test_res[i][0][0].detach().cpu().numpy() > 0.5
        ]).astype(np.uint8)),
        force2D=True
    )
    texturas.append(aux)

In [ ]:
# Extraemos las características con la función `.execute()` y las vamos almacenando en la lista de resultados.
results=[]
for i in range(80):
    texturas[i].enableAllFeatures()
    results.append(texturas[i].execute())

In [ ]:
# Construimos un dataframe con las características de texturas
df2 = pd.DataFrame(results)
df2['ID'] = [int(i) for i in train_dirs][0:80]
df2 = df2.set_index('ID')
df2

In [ ]:
# Características de primer orden
orden1=[]
for i in range(80):
    aux=radiomics.firstorder.RadiomicsFirstOrder(
        sitk.GetImageFromArray(test_img[i][0,0,:,:].reshape(1, 256, 256)), 
        sitk.GetImageFromArray(np.array([
            test_res[i][0][0].detach().cpu().numpy() > 0.5
        ]).astype(np.uint8)),
        force2D=True
    )
    orden1.append(aux)

In [ ]:
# Extraemos las características con la función `.execute()` y las vamos almacenando en la lista de resultados2.
results2=[]
for i in range(80):
    orden1[i].enableAllFeatures()
    results2.append(orden1[i].execute())

In [ ]:
# Construimos un dataframe con las características de primer orden
df3 = pd.DataFrame(results2)
df3['ID'] = [int(i) for i in train_dirs][0:80]
df3 = df3.set_index('ID')
df3

Una vez hemos extraído todas las características que queríamos, las juntamos todas en un solo dataframe con el que podamos trabajar. También normalizaremos los datos antes de aplicar ningún modelo.

In [ ]:
# Unimos los 3 dataframes que contienen las diferentes características de las imágenes
df = df.join(df2)
df = df.join(df3)
df

In [ ]:
# Normalizamos los datos 
scaler = MinMaxScaler()
df_norm  = scaler.fit_transform(df)


## Modelos de clasificación

A continuación, probamos varios clasificadores y los comparamos. 

In [ ]:
# Dividimos en train y test

x_tr, x_ts, y_tr, y_ts = train_test_split(df_norm, labels_train.to_numpy().squeeze(), test_size=0.2, random_state=0 ) 

In [ ]:
# Regresión Logística
clf = LogisticRegression(random_state=0)
clf.fit(x_tr, y_tr)
preds = clf.predict(x_ts)

print(classification_report(y_ts, preds))

In [ ]:
# Dibujamos la curva ROC
plot_roc_curve(clf,x_ts,y_ts)
plt.plot([0,1],[0,1])

In [ ]:
# Random Forest. 

from sklearn.ensemble import RandomForestClassifier 

RF = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
RF.fit(x_tr, y_tr)
preds_RF = RF.predict(x_ts)

print(classification_report(y_ts, preds_RF))

In [ ]:
# Dibujamos la curva ROC del RandomForest
plot_roc_curve(RF,x_ts,y_ts)
plt.plot([0,1],[0,1])

In [ ]:
# SVC

from sklearn.svm import SVC 

svc = SVC(kernel='rbf', gamma=1, random_state=0)
svc.fit(x_tr, y_tr)
preds_svc = svc.predict(x_ts)

print(classification_report(y_ts, preds_svc))

In [ ]:
# Dibujamos la curva ROC del SVC
plot_roc_curve(svc,x_ts,y_ts)
plt.plot([0,1],[0,1])

Vemos cuáles son las características más importantes según el modelo de Regresión Logística, es decir, cuáles son los biomarcadores más relevantes para la clasificación de las imágenes. Lo hacemos solo sobre este clasificador porque parecen el más relevante atendiendo a los resultados obtenidos de precisión y recall.

In [ ]:
# Regresión Logística : Importancia de características
# Usamos la función `permutation_importance` para calcular la importancia de cada característica. Básicamente lo que hace esta 
# función es barajar una de las características y ver como afecta eso al rendimiento del clasificador. Esto lo hace para cada 
# variable.
from sklearn.inspection import permutation_importance

importancias = permutation_importance(clf, x_tr, y_tr, random_state=0)

# Nos quedamos solo con las características más relevantes.
indx = np.where(importancias['importances_mean'] >= abs(0.006))
plt.bar(x= df.columns[indx], height=importancias['importances_mean'][indx])
plt.xticks(rotation=90)


Parece que las características más importantes a la hora de la clasificación son el Perímetro, el Diámetro Máximo y el Coeficiente de Correlación Máximo (MCC). Después de estas, tenemos otras variables como el Contraste, la Entropía y la Diferencia de Varianzas que están un nivel por debajo en cuánto a la importancia.

## Conclusiones

A nosotros nos interesa que el modelo sea capaz de acertar el máximo de casos positivos, es decir, que acierte el máximo de casos en los que hay un tumor en el órgano sobre el total de casos con tumor. Esto se corresponde con que el modelo tenga un recall alto. Además también querremos que de todos los casos en los que decimos que hay un tumor acertemos el máximo, es decir que tenga una precisión alta también. 

En principio, nos interesa más lo dicho anteriormente sobre la clase 1, pero tampoco podemos dejar de lado la clase 0. Por tanto, lo interesante será un recall y una precisión alta en ambas clases. 

Observando las métricas de los tres modelos propuestos juntos con sus curvas ROC, el modelo de Regresión Logística nos parece el mejor puesto que tiene los mejores valores de recall y precisión para la clase 1 (que es la más importante de detectar a priori) y, además, también tiene los mejores resultados para la clase 0. 

Comparando con el resto de modelos, vemos que el SVC es peor en general tanto para la clase 0 como para la clase 1, y que el Random Forest tiene los peores resultados para la clase 1. Por esto y por lo anterior, la regresión logística funcionaría mejor en este caso de clasificación.



## P.S. Warning
The segmentation doesn't not working properly on all images due to the different tumor modalities. The model was trained on low-grade tumors, so be aware. Let's see the examples

#### For this type of broken segmentation like in '00009' we could perform a little trick to fix. 
We need to multiply mask to the empty space of the original image

In [ ]:
im = get_img(6)
test_img = np.array([np.moveaxis(np.array(im.resize((256, 256))), -1, 0)])
test_res = model(torch.Tensor(test_img))

f, axarr = plt.subplots(1,5, figsize=(20, 20))
axarr[0].imshow(test_img[0, 0])
axarr[1].imshow(test_img[0, 1])
axarr[2].imshow(test_img[0, 2])
axarr[3].imshow(test_res[0][0].detach().cpu().numpy() > 0.5)
axarr[4].imshow((test_res[0][0].detach().cpu().numpy() > 0.5) * (test_img[0, 1] != 0))

#### For the "00003" image the broken segmentation could be due to the bad registration and this trick won't work

In [ ]:
im = get_img(2)
test_img = np.array([np.moveaxis(np.array(im.resize((256, 256))), -1, 0)])
test_res = model(torch.Tensor(test_img))

f, axarr = plt.subplots(1,5, figsize=(20, 20))
axarr[0].imshow(test_img[0, 0])
axarr[1].imshow(test_img[0, 1])
axarr[2].imshow(test_img[0, 2])
axarr[3].imshow(test_res[0][0].detach().cpu().numpy() > 0.5)
axarr[4].imshow((test_res[0][0].detach().cpu().numpy() > 0.5) * (test_img[0, 1] != 0))

#### Let's see more examples of broken segmentation

In [ ]:
im = get_img(9)
test_img = np.array([np.moveaxis(np.array(im.resize((256, 256))), -1, 0)])
test_res = model(torch.Tensor(test_img))

f, axarr = plt.subplots(1,5, figsize=(20, 20))
axarr[0].imshow(test_img[0, 0])
axarr[1].imshow(test_img[0, 1])
axarr[2].imshow(test_img[0, 2])
axarr[3].imshow(test_res[0][0].detach().cpu().numpy() > 0.5)
axarr[4].imshow((test_res[0][0].detach().cpu().numpy() > 0.5) * (test_img[0, 1] != 0))

In [ ]:
im = get_img(20)
test_img = np.array([np.moveaxis(np.array(im.resize((256, 256))), -1, 0)])
test_res = model(torch.Tensor(test_img))

f, axarr = plt.subplots(1,5, figsize=(20, 20))
axarr[0].imshow(test_img[0, 0])
axarr[1].imshow(test_img[0, 1])
axarr[2].imshow(test_img[0, 2])
axarr[3].imshow(test_res[0][0].detach().cpu().numpy() > 0.5)
axarr[4].imshow((test_res[0][0].detach().cpu().numpy() > 0.5) * (test_img[0, 1] != 0))

In [ ]:
im = get_img(24)
test_img = np.array([np.moveaxis(np.array(im.resize((256, 256))), -1, 0)])
test_res = model(torch.Tensor(test_img))

f, axarr = plt.subplots(1,5, figsize=(20, 20))
axarr[0].imshow(test_img[0, 0])
axarr[1].imshow(test_img[0, 1])
axarr[2].imshow(test_img[0, 2])
axarr[3].imshow(test_res[0][0].detach().cpu().numpy() > 0.5)
axarr[4].imshow((test_res[0][0].detach().cpu().numpy() > 0.5) * (test_img[0, 1] != 0))